In [ ]:
%pip install duckdb
%pip install jupysql
%pip install duckdb-engine

import datetime
import numpy as np
import pandas as pd
import pickle
import os

from dotenv import load_dotenv

from holidays_feature.holidays_feature import add_ukrainian_holidays
from holidays_feature.holidays_feature import add_russian_holidays
from eclipses_feature.eclipses_feature import add_lunar_eclipses
from eclipses_feature.eclipses_feature import add_solar_eclipses

import duckdb

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

load_dotenv("../.path_env")

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [ ]:
REPORTS_DATA_FILE = "./results/tfidf.csv"

# specifying the input folder
REPORTS_DATA_FILE = "../" + os.getenv("ISW_TF_IDF_RESULT")
OUTPUT_FOLDER = "results"
ISW_OUTPUT_DATA_FILE = "all_isw.csv"
WEATHER_EVENTS_OUTPUT_DATA_FILE = "all_hourly_weather_events.csv"

MODEL_FOLDER = "model"

tfidf_transformer_model = "tfidf_transformer"
count_vectorizer_model = "count_vectorizer"

tfidf_transformer_version = "v1"
count_vectorizer_version = "v1"

In [ ]:
def isNaN(num):
    return num != num

## reading data

In [ ]:
df_isw = pd.read_csv(f"{REPORTS_DATA_FILE}", sep=",")

In [ ]:
df_isw.head(5)

## preparing ISW reports

## reading models

In [ ]:
# #load the content
# tfidf = pickle.load(open(f"{MODEL_FOLDER}/{tfidf_transformer_model}_{tfidf_transformer_version}.pkl", "rb"))
# cv = pickle.load(open(f"{MODEL_FOLDER}/{count_vectorizer_model}_{count_vectorizer_version}.pkl", "rb"))

In [ ]:
# df_isw['keywords'] = df_isw['text_preprocessed_lemm'].apply(lambda x: tf_idf.conver_doc_to_vector(x,cv,tfidf))

In [ ]:
# df_isw.head(5)

In [ ]:
df_isw["date_datetime"] = pd.to_datetime(df_isw["Date"])

In [ ]:
df_isw["date_tomorrow_datetime"] = df_isw["date_datetime"].apply(
    lambda x: x + datetime.timedelta(days=1)
)
df_isw["event_time"] = np.nan

In [ ]:
df_isw = df_isw.rename(columns={"date_datetime": "report_date"})
df_isw.to_csv(f"{OUTPUT_FOLDER}/{ISW_OUTPUT_DATA_FILE}", sep=";", index=False)

In [ ]:
# Add holidays data to df_isw
add_ukrainian_holidays(df_isw, day_datetime_column='report_date', column_name='ukrainian_holiday')
add_russian_holidays(df_isw, day_datetime_column='report_date', column_name='russian_holiday')
# df_isw.loc[df_isw['ukrainian_holiday'] == 1]

In [ ]:
# df_isw.where(df_isw['ukrainian_holiday'] == 1)
# df_isw.sample(5)
# df_isw.loc[df_isw['ukrainian_holiday'] == 0].shape
# df_isw.shape
# df_isw.loc[df_isw['russian_holiday'] == 1]
# df_isw.loc[df_isw['ukrainian_holiday'] == 1]

## prepare events data

In [ ]:
EVENTS_DATA_FILE = "../" + os.getenv("ALARMS_DATA_FILE")

In [ ]:
df_events = pd.read_csv(f"{EVENTS_DATA_FILE}", sep=";")

In [ ]:
df_events_v2 = df_events.drop(["id", "region_id"], axis=1)
df_events_v2["event_time"] = np.nan

In [ ]:
df_events_v2.head(5)
df_events_v2.shape

In [ ]:
# df_events_v2["start_time"] = df_events_v2.apply(lambda x: x["start"] if not isNaN(x["start"]) else x["event_time"] , axis=1)
# df_events_v2["end_time"] = df_events_v2.apply(lambda x: x["end"] if not isNaN(x["end"]) else x["event_time"], axis=1)

In [ ]:
df_events_v2["start_time"] = pd.to_datetime(df_events_v2["start"])
df_events_v2["end_time"] = pd.to_datetime(df_events_v2["end"])
df_events_v2["event_time"] = pd.to_datetime(df_events_v2["event_time"])

In [ ]:
df_events_v2["start_hour"] = df_events_v2["start_time"].dt.floor("H")
df_events_v2["end_hour"] = df_events_v2["end_time"].dt.ceil("H")
df_events_v2["event_hour"] = df_events_v2["event_time"].dt.round("H")

In [ ]:
df_events_v2["start_hour"] = df_events_v2.apply(
    lambda x: x["start_hour"] if not isNaN(x["start_hour"]) else x["event_hour"], axis=1
)
df_events_v2["end_hour"] = df_events_v2.apply(
    lambda x: x["end_hour"] if not isNaN(x["end_hour"]) else x["event_hour"], axis=1
)

In [ ]:
df_events_v2["day_date"] = df_events_v2["start_time"].dt.date

df_events_v2.head(10)

df_events_v2["start_hour_datetimeEpoch"] = df_events_v2["start_hour"].apply(
    lambda x: int(x.timestamp()) if not isNaN(x) else None
)
df_events_v2["end_hour_datetimeEpoch"] = df_events_v2["end_hour"].apply(
    lambda x: int(x.timestamp()) if not isNaN(x) else None
)

# df_events_v2.head(10)

In [ ]:
# df_events_v2[~(df_events_v2["type"]=="alarm")].shape

In [ ]:
# df_events_v2[~(df_events_v2["type"]=="alarm")].head(5)

In [ ]:
# df_events_v2[df_events_v2["type"]=="alarm"].shape

## prepare weather

In [ ]:
WEATHER_DATA_FILE = "../" + os.getenv("WEATHER_DATA_FILE")

In [ ]:
df_weather = pd.read_csv(f"{WEATHER_DATA_FILE}")
df_weather["day_datetime"] = pd.to_datetime(df_weather["day_datetime"])

In [ ]:
df_weather.shape

In [ ]:
df_weather.head(15)

In [ ]:
# len(clmns)

In [ ]:
# exclude
weather_exclude = [
    "day_feelslikemax",
    "day_feelslikemin",
    "day_sunriseEpoch",
    "day_sunsetEpoch",
    "day_description",
    "city_latitude",
    "city_longitude",
    "city_address",
    "city_timezone",
    "city_tzoffset",
    "day_feelslike",
    "day_precipprob",
    "day_snow",
    "day_snowdepth",
    "day_windgust",
    "day_windspeed",
    "day_winddir",
    "day_pressure",
    "day_cloudcover",
    "day_visibility",
    "day_severerisk",
    "day_conditions",
    "day_icon",
    "day_source",
    "day_preciptype",
    "day_stations",
    "hour_icon",
    "hour_source",
    "hour_stations",
    "hour_feelslike",
]

In [ ]:
# new_list = [x for x in clmns if (x not in weather_exclude)]
# new_list

In [ ]:
df_weather_v2 = df_weather.drop(weather_exclude, axis=1)

In [ ]:
df_weather_v2["city"] = df_weather_v2["city_resolvedAddress"].apply(
    lambda x: x.split(",")[0]
)
df_weather_v2["city"] = df_weather_v2["city"].replace(
    "Хмельницька область", "Хмельницький"
)

In [ ]:
df_weather_v2.head(5)

In [ ]:
df_weather_v2.shape

## merging data

In [ ]:
REGIONS_DATA_FOLDER = "../external_data/additions"
REGIONS_DATA_FILE = "regions.csv"
df_regions = pd.read_csv(f"{REGIONS_DATA_FOLDER}/{REGIONS_DATA_FILE}")

In [ ]:
df_regions.head(5)

In [ ]:
df_weather_reg = pd.merge(
    df_weather_v2, df_regions, left_on="city", right_on="center_city_ua"
)

In [ ]:
df_weather_reg.head(10)

In [ ]:
df_weather_reg.shape

In [ ]:
df_weather_v2.shape

### Merging weather and events

In [ ]:
# df_events_v2["start_hour_datetimeEpoch"] = df_events_v2['start_hour'].apply(lambda x: int(x.strftime('%s'))  if not isNaN(x) else 0)
# df_events_v2["end_hour_datetimeEpoch"] = df_events_v2['end_hour'].apply(lambda x: int(x.strftime('%s'))  if not isNaN(x) else 0)

In [ ]:
df_events_v2.dtypes

In [ ]:
df_events_v2.shape

In [ ]:
df_events_v2.head(10)

In [ ]:
# df_events_v2_sample = df_events_v2.sample(10)
# df_events_v2_sample.shape

events_dict = df_events_v2.to_dict("records")
events_by_hour = []

In [ ]:
events_dict[0]

In [ ]:
for event in events_dict:
    for d in pd.date_range(start=event["start_hour"], end=event["end_hour"], freq="1H"):
        et = event.copy()
        et["hour_level_event_time"] = d
        events_by_hour.append(et)

In [ ]:
df_events_v3 = pd.DataFrame.from_dict(events_by_hour)
df_events_v3["hour_level_event_datetimeEpoch"] = df_events_v3[
    "hour_level_event_time"
].apply(lambda x: int(x.timestamp()) if not isNaN(x) else None)

In [ ]:
df_events_v3.shape

In [ ]:
df_events_v3.head(15)

In [ ]:
df_weather_reg.head(5)

In [ ]:
df_weather_reg.shape

In [ ]:
df_events_v3.head(10)
df_weather_reg.head(10)

In [ ]:
df_events_v4 = df_events_v3.copy().add_prefix("event_")
df_events_v4.head(10)

In [ ]:
df_weather_v4 = df_weather_reg.merge(
    df_events_v4,
    how="left",
    left_on=["region_alt", "hour_datetimeEpoch"],
    right_on=["event_region_title", "event_hour_level_event_datetimeEpoch"],
)

In [ ]:
# Alarm data
print(df_weather_v4.loc[~ isNaN(df_weather_v4['event_start'])].shape)
print(df_weather_v4.loc[isNaN(df_weather_v4['event_start'])].shape)
df_weather_v4['is_alarm'] = df_weather_v4.apply(lambda x: 0 if isNaN(x['event_start']) else 1, axis=1)
no_alarms = df_weather_v4.loc[df_weather_v4['is_alarm'] == 0].size
alarms = df_weather_v4.loc[df_weather_v4['is_alarm'] == 1].size
print(f"Alarm chane: {alarms / df_weather_v4.size}")
print(f"No alarm: {no_alarms / df_weather_v4.size}")
# df_weather_v4.sample(5)

In [ ]:
# Merge isw data to df_weather_v4
df_weather_v5 = pd.merge(
    df_weather_v4, df_isw[['Keywords','report_date', 'date_tomorrow_datetime', 'ukrainian_holiday', 'russian_holiday']], left_on="day_datetime", right_on="report_date"
)
df_weather_v5.head(5)

In [ ]:
# Add holidays to df_weather_v4
add_ukrainian_holidays(df_weather_v5, day_datetime_column='day_datetime', column_name='ukrainian_holiday')
add_russian_holidays(df_weather_v5, day_datetime_column='day_datetime', column_name='russian_holiday')
add_lunar_eclipses(df_weather_v5, day_datetime_column='day_datetime')
add_solar_eclipses(df_weather_v5, day_datetime_column='day_datetime')


df_weather_v5.head(5)

# print(df_weather_v4.shape)
# print(df_weather_v4.loc[df_weather_v4['ukrainian_holiday'] == 1].shape)
# print(df_weather_v4.loc[df_weather_v4['russian_holiday'] == 1].shape)
# #
# print(df_weather_v5.shape)
# print(df_weather_v5.loc[df_weather_v5['ukrainian_holiday'] == 1].shape)
# print(df_weather_v5.loc[df_weather_v5['russian_holiday'] == 1].shape)

df_weather_v4["hour_conditions"] = pd.Categorical(df_weather_v4['hour_conditions'])
df_weather_v4["hour_conditions_code"] = df_weather_v4["hour_conditions"].cat.codes
#
df_weather_v5["hour_conditions"] = pd.Categorical(df_weather_v5['hour_conditions'])
# df_weather_v5["hour_conditions"].astype('category')
# weather_condition_categories = df_weather_v5["hour_conditions"].cat.categories.values
# weather_condition_categories
df_weather_v5["hour_conditions_code"] = df_weather_v5["hour_conditions"].cat.codes
df_weather_v5[["hour_conditions", "hour_conditions_code"]].head(5)

df_weather_v6 = None


### Handle categorical data

In [ ]:
df_weather_v5.head(5)

In [ ]:
%%sql
df_weather_v6 << select df.*, coalesce(alarmed_count.alarmed_regions_count, 0) as alarmed_regions_count
from df_weather_v5 df
         left join (select out.day_datetimeEpoch,
                                            out.hour_datetimeEpoch,
                                            count(*) as alarmed_regions_count
                                    from df_weather_v5 out left join df_weather_v5 inn
                                         on out.day_datetimeEpoch = inn.day_datetimeEpoch and out.hour_datetimeEpoch = inn.hour_datetimeEpoch
                                    where inn.is_alarm = 1 and inn.hour_datetimeEpoch
                                                               between out.event_start_hour_datetimeEpoch and out.event_start_hour_datetimeEpoch
                                    group by out.day_datetimeEpoch, out.hour_datetimeEpoch) as alarmed_count
        on df.day_datetimeEpoch = alarmed_count.day_datetimeEpoch and df.hour_datetimeEpoch = alarmed_count.hour_datetimeEpoch;


In [ ]:
df_weather_v6.head(10)

### Save final merged dataframe

In [ ]:
df_weather_v4.to_csv(
    f"{OUTPUT_FOLDER}/{WEATHER_EVENTS_OUTPUT_DATA_FILE}", sep=";", index=False
)

In [ ]:
# Uncomment to save final merged dataset ~4G
# df_weather_v5.to_csv(
#     f"{OUTPUT_FOLDER}/all_hourly_weather_events_final.csv", sep=";", index=False
# )

### Test LinearRegression

In [ ]:
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression

df_final = df_weather_v6

# Fill NaN values
df_final[['event_all_region']] = df_final[['event_all_region']].fillna(value=0)

# Separating the data into independent and dependent variables
# Converting each dataframe into a numpy array
X = np.array(df_final[['region_id', 'event_all_region', 'day_datetimeEpoch', 'hour_datetimeEpoch', 'ukrainian_holiday', 'russian_holiday', 'hour_temp', 'hour_snow', 'hour_visibility', 'hour_conditions_code', 'lunar_eclipse', 'solar_eclipse', 'moonphased_eclipse','alarmed_regions_count']])
y = np.array(df_final['is_alarm'])

# Dropping any rows with Nan values
# df_final.dropna(inplace = True)

# Splitting the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=101)

# LinerRegression
regr = LinearRegression()
#
regr.fit(X_train, y_train)
#
print(regr.score(X_test, y_test))
# df_weather_v4 -      0.7357812097367479
# df_weather_v5 -      0.728836911878402
# hours with no alarm - 0.7475497734309323


In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import SGDRegressor

sgd = SGDClassifier(loss='log_loss', max_iter=1000, tol=1e-3, random_state=42)
sgd.fit(X_train, y_train)

# Evaluate the model
accuracy = sgd.score(X_test, y_test)
print(accuracy)



In [ ]:
%pip install matplotlib
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
# Predict the labels for the test set
y_pred = sgd.predict(X_test)

# Calculate the confusion matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)


In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix,
                             display_labels=sgd.classes_)
disp.plot()

In [ ]:
# Creating Our First Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
accuracy = clf.score(X_test, y_test)
print(accuracy)
y_pred = clf.predict(X_test)
confusion_matrix = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix,
                             display_labels=sgd.classes_)
disp.plot()